# Setup

In [ ]:
!pip install elevenlabs num2words

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 26.9 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=cfc3e3ccc357965a63f4deb1c231c6f81ecfc74c1515f2ff34dfb74b41a70ffe
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
from elevenlabs import generate, play, clone, save
from elevenlabs.api import Voices
from elevenlabs.simple import set_api_key
import os
from google.colab import files
import shutil
import re
from num2words import num2words

In [ ]:
set_api_key('be67b217717a1a7e631681d0c85395d4')

In [ ]:
voices = Voices.from_api()

Verify what voice you want.

In [ ]:
voices[-2]

Voice(voice_id='J16nzqxFhkQhOadf18wF', name='NVI_22_v2', category='cloned', description='', labels={}, samples=[VoiceSample(sample_id='1MO22xv5ZJ0yaSSNjiXG', file_name='19_Psalm 089_3.mp3', mime_type='audio/mpeg', size_bytes=9600044, hash='085a055239082dc71d41c597b68dd0bf'), VoiceSample(sample_id='4iQIUkX3sAeDP6hBIwsv', file_name='19_Psalm 018_5.mp3', mime_type='audio/mpeg', size_bytes=9600044, hash='980c7d500c8525a6267cb944ce945a69'), VoiceSample(sample_id='5SjHdln4EY57jNdEd5WN', file_name='19_Psalm 018_0.mp3', mime_type='audio/mpeg', size_bytes=9600044, hash='82d3c5894cc0d388ff3d002e70d4fc16'), VoiceSample(sample_id='6VHzNL47bkrQTMRtbkwq', file_name='19_Psalm 078_3.mp3', mime_type='audio/mpeg', size_bytes=9600044, hash='fc4a7a1a52dfef423e7a426017016853'), VoiceSample(sample_id='7UDu5Eh7ly1KXjhj6BAd', file_name='19_Psalm 018_2.mp3', mime_type='audio/mpeg', size_bytes=9600044, hash='d2043ffbc310ee62f3d3ad98093def14'), VoiceSample(sample_id='BcAzaTeCi3ocSFomkKB1', file_name='19_Psalm 11

# Normalize Text

There are some passages where numbers are represented as digits instead of words (i.e. 1 instead of 'uno'). This causes issues with ElevenLabs as it reads these characters in English with a Spanish accent. The following function fixes that issue.

In [ ]:
def normalize_text(text):
  text = text.replace('«','"')
  text = text.replace('»','"')
  text = text.replace('\t', '')
  text = text.replace('\xa0\xa0','')
  normalized_text = convert_numerals(text)

  return normalized_text

In [ ]:
def convert_numerals(text):
    def replace_numerals(match):
        num = match.group(0).replace(',', '')
        return num2words(int(num), lang='es')

    pattern = r'\b\d{1,3}(,\d{3})*\b'
    converted_text = re.sub(pattern, replace_numerals, text)
    return converted_text

Select the appropriate voice and set parameters.

In [ ]:
custom_voice = voices[-2]
custom_voice.settings.stability = 1.0
custom_voice.settings.similarity_boost = 1.0

# Generate Audio

In [ ]:
book = 'JHN'

root_directory = f"/content/{book}"
os.makedirs(root_directory)
os.makedirs('/content/chapter_audio')

In [ ]:
for chapter in os.listdir(root_directory):
    chapter_directory = os.path.join(root_directory, chapter)

    try:
      with open(chapter_directory, "r") as file:
          chapter_text = file.read()

      # normalize to eliminate tabs and characters that cause issues for EL
      normalized_text = normalize_text(chapter_text)

      print(f'Generating audio for: {chapter}')

      text_length = normalized_text.split(' ')

      # can't have more than 5400 characters per generate request & 900 words seems like the best cutoff point
      if len(text_length) > 900:
        chunk_number = 1
        start = 0
        end = len(text_length)
        step = 900
        for i in range(start, end, step):
            x = i
            chunk = ' '.join(text_length[x:x+step])
            verse_audio = generate(
            text= chunk,
            voice= custom_voice,
            model='eleven_multilingual_v1'
            )

            name = chapter.split('.')

            save(verse_audio, f'/content/chapter_audio/{name[0]}_{chunk_number}.wav')
            chunk_number += 1
      else:
        verse_audio = generate(
        text= normalized_text,
        voice= custom_voice,
        model='eleven_multilingual_v1'
        )

        name = chapter.split('.')

        save(verse_audio, f'/content/chapter_audio/{name[0]}.wav')

    except IsADirectoryError:
      pass

print('Completed Audio')

Generating audio for: JHN_1.txt
Completed Audio


#Download

In [ ]:
folder_path = f'/content/chapter_audio'
zip_filename = f'{book}'
file = shutil.make_archive(zip_filename, 'zip', folder_path)
files.download(file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Delete Directories

In [ ]:
shutil.rmtree('/content/chapter_audio')
shutil.rmtree(f'/content/{book}')